This notebook is for testing the application. Read the comments. You can play around with it. 

Make sure to have your api key and your model name configured in `services/config/personal_constants.py`

In [ ]:
# importing the class from a different file
from services.querier.querier import DbQuerier

# instantiating the class
test_querier = DbQuerier(sample_db_loc='data/PostNL_SQLite.sqlite')

# do some configurations
test_querier.remove_all_training_data() # if there was any, now there is not
test_querier.train_model(train_on_documentation=True, 
                         train_on_ddl=True, 
                         train_on_sql=True, 
                         train_on_question_sql_pairs=False)

In [ ]:
# showing the training data that is in the model
test_querier.training_data

In [ ]:
# some code to play around with
question = """
Wat is de verhouding tussen grote, middelgrote en kleine paketten?
"""
sql_code = test_querier.generate_sql(question)
print(sql_code)

sample_data = test_querier.generate_sample_data(sql_query=sql_code)
sample_data